# I

In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate, Activation, UpSampling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import Sequence
from tensorflow.keras.applications import ResNet50
from skimage.io import imread
from skimage.transform import resize

# Custom Sobel Loss functions
def dice_loss(y_true, y_pred):
    numerator = 2 * tf.reduce_sum(y_true * y_pred, axis=[1, 2, 3])
    denominator = tf.reduce_sum(y_true + y_pred, axis=[1, 2, 3])
    return 1 - numerator / denominator

def unsupervised_loss(y_pred):
    return tf.reduce_mean(tf.image.total_variation(y_pred))

@tf.keras.utils.register_keras_serializable()
def combined_loss(y_true, y_pred):
    supervised_mask = tf.expand_dims(y_true[..., -1], axis=-1)
    unsupervised_mask = 1 - supervised_mask
    y_true_masked = y_true[..., :-1] * supervised_mask
    y_pred_masked_supervised = y_pred * supervised_mask
    y_pred_masked_unsupervised = y_pred * unsupervised_mask

    supervised_loss = dice_loss(y_true_masked, y_pred_masked_supervised)
    unsupervised_loss_value = unsupervised_loss(y_pred_masked_unsupervised)

    return supervised_loss + 0.1 * unsupervised_loss_value

def conv_block(input_tensor, num_filters):
    x = Conv2D(num_filters, (3, 3), padding='same')(input_tensor)
    x = Activation('relu')(x)
    x = Conv2D(num_filters, (3, 3), padding='same')(x)
    x = Activation('relu')(x)
    return x

# Data Generator for training
class DataGenerator(Sequence):
    def __init__(self, image_dir, mask_dir, batch_size=8, image_size=(256, 256), num_classes=4, shuffle=True):
        self.image_filenames = [os.path.join(image_dir, x) for x in os.listdir(image_dir) if x.endswith('.png')]
        self.mask_filenames = [os.path.join(mask_dir, x) for x in os.listdir(mask_dir) if x.endswith('.png')]
        self.batch_size = batch_size
        self.image_size = image_size
        self.num_classes = num_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.image_filenames) / self.batch_size))

    def __getitem__(self, index):
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        image_filenames_temp = [self.image_filenames[k] for k in indexes]
        mask_filenames_temp = [self.mask_filenames[k] for k in indexes]
        X, y = self.__generate_data(image_filenames_temp, mask_filenames_temp)
        return X, y

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.image_filenames))
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __generate_data(self, image_filenames_temp, mask_filenames_temp):
        X = np.empty((self.batch_size, *self.image_size, 3))
        y = np.empty((self.batch_size, *self.image_size, self.num_classes + 1))
        for i, (img_path, mask_path) in enumerate(zip(image_filenames_temp, mask_filenames_temp)):
            img = imread(img_path)
            mask = imread(mask_path, as_gray=True)
            mask = resize(mask, self.image_size, order=0, preserve_range=True)

            # Ensure no unexpected values are in the mask
            unique_values = np.unique(mask)
            if np.any(unique_values >= self.num_classes):
                print(f"Unexpected values found in {mask_path}: {unique_values}")
                mask = np.where(mask >= self.num_classes, 0, mask)  # Remap unexpected values

            if img.ndim == 2:
                img = np.stack((img,)*3, axis=-1)
            elif img.ndim == 3 and img.shape[-1] == 1:
                img = np.concatenate([img]*3, axis=-1)

            img = resize(img, self.image_size, preserve_range=True)
            X[i, ] = img
            y[i, ..., :-1] = tf.keras.utils.to_categorical(mask, num_classes=self.num_classes)
            y[i, ..., -1] = np.any(mask > 0, axis=-1)
        return X, y





# U-Net Model with ResNet50 encoder
def build_model(input_shape, num_classes):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(input_shape[0], input_shape[1], 3))
    layer_dict = dict([(layer.name, layer.output) for layer in base_model.layers])
    skip_connection_names = ["conv1_relu", "conv2_block3_out", "conv3_block4_out", "conv4_block6_out"]
    skip_connections = [layer_dict[name] for name in skip_connection_names]
    encoder_output = layer_dict["conv5_block3_out"]

    # Decoder
    x = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(encoder_output)
    x = concatenate([x, skip_connections[3]])  # Skip connection from conv4
    x = conv_block(x, 512)
    x = UpSampling2D((2, 2))(x)
    x = concatenate([x, skip_connections[2]])  # Skip connection from conv3
    x = conv_block(x, 256)
    x = UpSampling2D((2, 2))(x)
    x = concatenate([x, skip_connections[1]])  # Skip connection from conv2
    x = conv_block(x, 128)
    x = UpSampling2D((2, 2))(x)
    x = concatenate([x, skip_connections[0]])  # Skip connection from conv1
    x = conv_block(x, 64)
    outputs = Conv2D(num_classes, (1, 1), activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=outputs)
    return model

# Model Compilation and Training
input_shape = (256, 256, 3)  # Adjusted to 3 channels
num_classes = 4  # For three organs and background
model = build_model(input_shape, num_classes)
model.compile(optimizer='adam', loss=combined_loss)

# Define paths to your data directories
image_dir = '/Users/arahjou/Downloads/Medical_seg/dataset_UWM_GI_Tract_train_valid/train/masks'
mask_dir = '/Users/arahjou/Downloads/Medical_seg/dataset_UWM_GI_Tract_train_valid/train/images'

train_gen = DataGenerator(image_dir, mask_dir, batch_size=8, image_size=(256, 256), num_classes=num_classes)

# Train the model
model.fit(train_gen, epochs=3)


IndexError: index 4 is out of bounds for axis 1 with size 4

# II

Shifting towards a more unsupervised approach for segmentation tasks usually involves leveraging techniques like autoencoders, generative adversarial networks (GANs), or self-supervised learning methods that do not rely on labeled data. However, maintaining some level of supervised learning can help guide the model, particularly in complex tasks like medical image segmentation. Here, I'll adjust the model to use a mix of self-supervised learning for feature extraction and unsupervised learning for segmentation consistency, while significantly reducing the reliance on labeled data.

### Approach:
1. **Autoencoder as a Base**: We'll use an autoencoder for learning useful representations of the data in an unsupervised manner. The encoder part will learn to compress the image data into a latent space, and the decoder will learn to reconstruct the image from this compressed representation.

2. **Reconstruction Loss**: We will use a reconstruction loss which encourages the autoencoder to produce outputs as close as possible to its inputs, learning useful features in the process without needing labels.

3. **Segmentation as a Side Task**: After training the autoencoder, we can attach a segmentation head to the encoded features to perform segmentation, optimizing this task with a separate unsupervised loss, such as consistency or entropy minimization over the predicted segmentation maps.


### Explanation:
- **Autoencoder**: This model is trained purely unsupervised. It learns to compress and decompress the input images, capturing their essential features in the process.
- **Data Generator**: Modified to support autoencoder training, where both inputs and outputs are the same image (self-reconstruction).

### Next Steps:
- **Segmentation as Side Task**: After training the autoencoder, explore adding a segmentation head to the encoder part and train it using an unsupervised loss (e.g., consistency loss across different augmentations of the same image).

This setup emphasizes learning from unlabeled data, significantly reducing reliance on labeled data and moving closer to an unsupervised learning paradigm.

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate, Activation, UpSampling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import Sequence
from skimage.io import imread
from skimage.transform import resize

# Autoencoder for unsupervised feature learning
def build_autoencoder(input_shape):
    inputs = Input(input_shape)
    # Encoder
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    encoded = MaxPooling2D((2, 2), padding='same')(x)

    # Decoder
    x = Conv2DTranspose(256, (3, 3), strides=(2, 2), padding='same', activation='relu')(encoded)
    x = Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same', activation='relu')(x)
    x = Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same', activation='relu')(x)
    decoded = Conv2D(input_shape[2], (3, 3), activation='sigmoid', padding='same')(x)
    
    autoencoder = Model(inputs, decoded)
    autoencoder.compile(optimizer='adam', loss='mse')
    return autoencoder

# Data Generator for training
class DataGenerator(Sequence):
    def __init__(self, image_dir, batch_size=8, image_size=(256, 256), shuffle=True):
        self.image_filenames = [os.path.join(image_dir, x) for x in os.listdir(image_dir) if x.endswith('.png')]
        self.batch_size = batch_size
        self.image_size = image_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.image_filenames) / self.batch_size))

    def __getitem__(self, index):
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        image_filenames_temp = [self.image_filenames[k] for k in indexes]
        X = self.__generate_data(image_filenames_temp)
        return X, X  # Input and output are the same for autoencoder

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.image_filenames))
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __generate_data(self, image_filenames_temp):
        X = np.empty((self.batch_size, *self.image_size, 3))
        for i, img_path in enumerate(image_filenames_temp):
            img = resize(imread(img_path), self.image_size, preserve_range=True)
            X[i,] = img / 255.0  # Normalize images
        return X

# Train the autoencoder
input_shape = (256, 256, 3)
autoencoder = build_autoencoder(input_shape)
image_dir = '/path/to/your/images'
train_gen = DataGenerator(image_dir, batch_size=8, image_size=(256, 256))

# Train autoencoder
autoencoder.fit(train_gen, epochs=10)

# Adding a segmentation head (optional)
# This would be your task to explore how to use the learned features for segmentation!


Certainly! After training the autoencoder to learn useful representations from the data, the next step involves attaching a segmentation head to the encoded features. This segmentation model will be trained to predict segmentation masks, but in this unsupervised or semi-supervised scenario, we can train it using consistency losses or pseudo-labeling approaches. Here, I'll introduce a simple unsupervised learning method for segmentation using a pseudo-labeling approach where we generate pseudo-labels based on the model's own predictions.

### Segmentation Head and Pseudo-label Training
For simplicity, we'll generate pseudo-labels from the high-confidence predictions of the segmentation model and use them as targets for further training. This approach assumes that predictions with high confidence are likely to be correct. You can further refine this by using more sophisticated consistency or entropy minimization techniques.

```python

```

### Explanation:
- **Segmentation Model**: This is built by extending the encoder part of the autoencoder with a new segmentation head.
- **Pseudo-labels**: During training, the model uses its own predictions as targets, refining these predictions iteratively.
- **Data Generator for Segmentation**: This now also generates pseudo-labels using the segmentation model's predictions to train the model in an unsupervised manner.

This setup allows you to train the segmentation model in a way that reduces reliance on labeled data, using the model's own confidence in its predictions to guide the learning process.

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, MaxPooling2D, concatenate, Activation, UpSampling2D
from tensorflow.keras.optimizers import Adam

# Build the complete model: Autoencoder + Segmentation head
def build_segmentation_model(autoencoder, num_classes):
    # Encoder uses the first part of the autoencoder
    encoder = autoencoder.input
    encoded = autoencoder.layers[-9].output  # This is an example, adjust based on your autoencoder design

    # Segmentation head
    x = Conv2DTranspose(256, (3, 3), strides=(2, 2), padding='same', activation='relu')(encoded)
    x = Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same', activation='relu')(x)
    x = Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same', activation='relu')(x)
    outputs = Conv2D(num_classes, (1, 1), activation='softmax')(x)

    model = Model(inputs=encoder, outputs=outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy')  # Use a suitable loss function
    return model

# Generate pseudo-labels for unsupervised training
def generate_pseudo_labels(model, data):
    predictions = model.predict(data)
    pseudo_labels = np.argmax(predictions, axis=-1)  # Get the class with the highest probability
    pseudo_labels = tf.keras.utils.to_categorical(pseudo_labels, num_classes=num_classes)  # Convert to one-hot
    return pseudo_labels

# Data Generator modified for segmentation training with pseudo-labels
class SegDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, image_dir, batch_size, image_size, num_classes, model=None, shuffle=True):
        self.image_filenames = [os.path.join(image_dir, x) for x in os.listdir(image_dir) if x.endswith('.png')]
        self.batch_size = batch_size
        self.image_size = image_size
        self.num_classes = num_classes
        self.model = model
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.image_filenames) / self.batch_size))

    def __getitem__(self, index):
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        image_filenames_temp = [self.image_filenames[k] for k in indexes]
        X = self.__generate_data(image_filenames_temp)
        if self.model:
            y = generate_pseudo_labels(self.model, X)  # Generate pseudo-labels using the model
        else:
            y = np.zeros((self.batch_size, *self.image_size, self.num_classes))  # Dummy labels if model is None
        return X, y

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.image_filenames))
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __generate_data(self, image_filenames_temp):
        X = np.empty((self.batch_size, *self.image_size, 3))
        for i, img_path in enumerate(image_filenames_temp):
            img = resize(imread(img_path), self.image_size, preserve_range=True)
            X[i,] = img / 255.0  # Normalize images
        return X

# Initialize and train the segmentation model with pseudo-labels
input_shape = (256, 256, 3)
num_classes = 4
autoencoder = build_autoencoder(input_shape)
segmentation_model = build_segmentation_model(autoencoder, num_classes)
image_dir = '/path/to/your/images'

# Train using the autoencoder for initial weights
train_gen = SegDataGenerator(image_dir, batch_size=8, image_size=(256, 256), num_classes=num_classes, model=segmentation_model)
segmentation_model.fit(train_gen, epochs=10)  # Adjust epochs as needed